# Stenoprotokoly - příprava dat
Web poslanecké sněmovny má velký archív stenoprotokolů. Můžete si třeba pročíst, jak se probíraly záležitosti [v roce 1920](http://www.psp.cz/eknih/1920ns/index.htm). Nás bude zajímat současná sněmovna, k níž stena najdeme [tady](http://www.psp.cz/eknih/2013ps/stenprot/index.htm). Naštěstí nemusíme data stahovat přímo v tomto formátu, PSP nabízí i zazipovaný archív, konkrétně [tady](http://www.psp.cz/eknih/2013ps/stenprot/zip/index.htm).

Probíhat to bude asi takto: stáhneme zipy, rozbalíme, rozparsujeme do jsonů, nasypeme do skladiště dle libosti. Bude třeba Python 3 a pár balíků. Většina ve standardní knihovně, mimo ní používám akorat PyQuery.

Je třeba dodělat:
- datum projevu
- rozparsovat jméno řečníka (prozatím funkční)

In [1]:
import urllib.request as ur
import urllib.parse as up
import urllib
# from pyquery import PyQuery as pq
import lxml.html
import os.path
import zipfile
import glob
from collections import OrderedDict
import json
from collections import Counter
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import string
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
base = 'http://www.psp.cz/eknih/'
zp_folder = './zip/'
zp_ext = '.zip'
st_folder = './html/'
js_folder = 'json/'

Stáhneme stránku se seznamem jednání a najdeme odkazy na jednání Poslanecké sněmovny

In [3]:
ht = lxml.html.parse(base).getroot()

In [4]:
links = []

libs = ht.cssselect('div#main-content')[0]
for elem_a, elem_b in zip(libs.cssselect('a'), libs.cssselect('a b')):
    if elem_b.text.endswith('Poslanecká sněmovna'):
        links.append(up.urljoin(base, elem_a.attrib['href']))

print(links)

['http://www.psp.cz/eknih/2017ps/index.htm', 'http://www.psp.cz/eknih/2013ps/index.htm', 'http://www.psp.cz/eknih/2010ps/index.htm', 'http://www.psp.cz/eknih/2006ps/index.htm', 'http://www.psp.cz/eknih/2002ps/index.htm', 'http://www.psp.cz/eknih/1998ps/index.htm', 'http://www.psp.cz/eknih/1996ps/index.htm']


Ze stránek jednotlivých jednání PS vezmeme odkazy na komprimované stenografy

In [5]:
comp_links = []

for ln in links:
    page = lxml.html.parse(ln).getroot().cssselect('div#main-content')[0]
    for elem_a in page.cssselect('a'):
        if elem_a.text:
            if 'komprimované' in elem_a.text:
                comp_links.append(up.urljoin(ln, elem_a.attrib['href']))

print(comp_links)

['http://www.psp.cz/eknih/2017ps/stenprot/zip/', 'http://www.psp.cz/eknih/2013ps/stenprot/zip/index.htm', 'http://www.psp.cz/eknih/2010ps/stenprot/zip/index.htm', 'http://www.psp.cz/eknih/2006ps/stenprot/zip/index.htm', 'http://www.psp.cz/eknih/2002ps/stenprot/zip/index.htm']


Stenoprotokoly jsou zazipované, ale jsou stále ve spousta balících. Stáhneme tedy stránku, na které jsou, projdeme odkazy na zipy a stáhneme ty.

In [6]:
if not os.path.exists('zip'): os.mkdir('zip')
page_num = 0
for comp_link in comp_links:
    page = lxml.html.parse(comp_link).getroot()
    for ln in page.cssselect('div#main-content a'):
        ln_attr_href = ln.attrib['href'].split('/')[-1]
        target = os.path.join(zp_folder, str(page_num) + '_' + ln_attr_href)
        if os.path.isfile(target): continue
        try:
            print('Retrieving ' + up.urljoin(comp_link, ln_attr_href) + ' ', end='')
            ur.urlretrieve(up.urljoin(comp_link, ln_attr_href), target)
            print('done.')
        except urllib.error.HTTPError as err:
            print('failed: {}'.format(err))
    page_num += 1

Retrieving http://www.psp.cz/eknih/2017ps/stenprot/zip/037schuz.zip done.
Retrieving http://www.psp.cz/eknih/2013ps/stenprot/zip/059schuz.zip failed: HTTP Error 404: Not Found
Retrieving http://www.psp.cz/eknih/2013ps/stenprot/zip/060schuz.zip failed: HTTP Error 404: Not Found
Retrieving http://www.psp.cz/eknih/2013ps/stenprot/zip/061schuz.zip failed: HTTP Error 404: Not Found


Rozbalíme (šlo by číst přímo z archivů, ale proč si to komplikovat, když to místa zabírá málo)

In [7]:
if not os.path.exists('html'): os.mkdir('html')

for arch in os.listdir(zp_folder):
    if arch.endswith(zp_ext):
        file_name = os.path.join(zp_folder, arch)
        target = os.path.join(st_folder, arch.split('.')[0])

        if os.path.exists(target): continue
        
        with zipfile.ZipFile(file_name) as zpf:
            print('Extracting ' + file_name + ' to ' + target + ' ', end='')
            try:
                zpf.extractall(target)
                print('done.')
            except OSError as err:
                os.rmdir(target)
                print('failed: {}'.format(err))

Extracting ./zip/2_042schuz.zip to ./html/2_042schuz failed: [Errno 22] Invalid argument
Extracting ./zip/0_037schuz.zip to ./html/0_037schuz done.


Převedeme do textu. Není tu moc rocket science, vesměs jde o:

- Každý odstavec je buďto pokračování projevu, nebo začátek nového.
- Začátek projevu poznáme tak, že začíná odkazem na profil řečníka. Hledat odkaz v textu je ale celkem riskantní, protože některé projevy (zvlášť v minulé Sněmovně) obsahovaly odkazy na hlasování. Tohle bude třeba ještě nějak ošetřit.
- Proslovy mohou přetékat mezi soubory. Na začátku souboru je tak třeba zjistit, zda jde o pokračování, nebo ne. Já to řeším tak, že konce souborů neřeším, celá jedna schůze je pro mě tok textu, bez ohledu na rozdělení mezi soubory.
- Jména řečníků jsou vč. jejich funkcí. To je nepříjemné ze dvou důvodů: jméno je moc dlouhé, zvlášť u facetování (filtrace v hledání), ale hlavně protože jméno pak není unikátní, protože funkce se mění, zvlášť mezi sněmovnami. Když pak budeme chtít seskupit spoustu let dohromady, budeme mít např. Bohuslava Sobotku jako ministra financí, poslance a premiéra. Bohužel rozdělení jména na funkce a vlastní jméno není úplně jednoznačné, bude to chtít vypsat počet variací a ošetřit to.
- EDIT: Pro současná dostupná data (říjen 2019) odstranení pozic funguje docela dobře. Stačí do proměnné "poz" vyjmenovat slova, kterými pozice končí a podle nic odkrojit začátek stringu. Problém nastane, až se některý z politiků bude jmenovat "Poslanec", "Senátor" atp.

In [8]:
if not os.path.exists(js_folder): os.mkdir(js_folder)
html_files = glob.glob('./html/*')

In [9]:
poz = 'Poslanec; PSP; Paní; Senátorka; Senátor; \
Poslankyně; mužů; Předsedající; práv; republiky; financí; prostředí; věcí; ČR'.split('; ')

def rm_position(name):    
    for p in poz:
        if p in name:
            name = name[name.rindex(p) + len(p) + 1:]
            
    return name

In [10]:
def write_json(nm, dt):
    fn = os.path.join(js_folder, '%s.json' % nm)
    
    with open(fn, 'w') as f:
        t = json.dump(dt, f, ensure_ascii=False, indent=2)

res = []
pid = 0
aut = None
tema = None
buf = []
for htmlf in html_files:
    buff = []
    fns = glob.glob(os.path.join(htmlf, 's*.htm'))
    for fn in fns:
        h = lxml.html.parse(fn).getroot()
        for p in h.cssselect('p'):
            pt = p.text_content().strip()
            if len(pt) == 0: continue
            pt = pt.replace('\xa0', ' ')
            
            od = p.find('a') # v textu je odkaz
            if od is None:
                buf += [pt]
                continue

            if len(buf) > 0:
                buff.extend([OrderedDict(id=pid, autor=aut, schuze=int(htmlf.split('/')[-1][:3]),\
                                         fn=fn, tema=tema, text='\n'.join(buf))])

            aut = rm_position(od.text.strip())
            buf = [pt[len(od.text)+1:].strip()] # pridame soucasny text (ale odseknem autora)
            pid += 1
    write_json(htmlf[-htmlf[::-1].find('/'):], buff)

print(htmlf)

./html/2_010schuz


## Kontrola odstranení pozic ze jmen

In [11]:
json_files = glob.glob('json/*.json')

In [12]:
auts = []
for fn in json_files:
    with open(fn) as f:
        dt = json.load(f)
    
    for el in dt:
        if el['autor'] is not None:
            auts.append(el['autor'])

Nejčastější mluvčí

In [13]:
Counter(auts).most_common()[:3]

[('Vojtěch Filip', 12595), ('Petr Gazdík', 8884), ('Jan Bartošek', 7854)]

Jména delší než dvě slova

In [14]:
[j for j in set(auts) if len(j.split(' ')) > 2]

['Augustin Karel Andrle Sylor',
 'Markéta Pekarová Adamová',
 'Jaroslava Pokorná Jermanová',
 'Zuzana Majerová Zahradníková',
 'Jana Mračková Vildumetzová',
 'Tomáš Jan Podivínský',
 'Zuzka Bebarová Rujbrová',
 'Hana Aulická Jírovcová']

## Větná tokenizace

Z json souborů vezmeme všechen text, změníme na malá písmena, odstraníme veškerou diakritiku a bílé znaky na začátku/konci vět.

Věty pak zapíšeme do souboru, kde každý řádek bude právě jedna věta.

Do souboru nezapisuji věty obsahující číslice.

In [15]:
def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

In [16]:
json_files = glob.glob('json/*.json')
translator = str.maketrans('', '', string.punctuation)
str_builder = ''

file = open('./vocabulary.txt', 'w+')

for fn in json_files:
    with open(fn) as f:
        jsf = json.load(f)
    
    for el in jsf:
        for s in sent_tokenize(el['text']):
            processed_str = s.lower().translate(translator).strip()
            if processed_str and not hasNumbers(processed_str):
                str_builder += processed_str + '\n'
    file.write(str_builder)
    str_builder = ''

file.close()